In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：为在线预测训练表格化二元分类模型的AutoML

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_binary_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_binary_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_binary_classification_online.ipynb">
      在Google Cloud笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK创建表格二元分类模型，并使用Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)模型进行在线预测。

### 数据集

此教程使用的数据集是[银行营销](https://pantheon.corp.google.com/storage/browser/_details/cloud-ml-tables-data/bank-marketing.csv)。 这个数据集不需要任何特征工程。您将在本教程中使用的数据集版本存储在公共云存储桶中。

### 目标

在本教程中，您将使用Vertex SDK从Python脚本创建一个AutoML表格二元分类模型，并部署用于在线预测。您也可以选择使用`gcloud`命令行工具或在线使用Cloud Console来创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到serving `Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

### 成本

这个教程使用了 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Google Cloud笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境的云存储指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简化的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端窗口中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端窗口中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装最新版本的Python版Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

安装了额外的软件包后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的谷歌云项目

**无论您使用的是哪个笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。您首次创建帐号时，可以获得一笔 $300 的免费信用额度，用于支付计算和存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您是在本地运行此笔记本，您需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并对以 `$` 为前缀的 Python 变量进行插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于笔记本的其余操作。 以下是支持Vertex AI的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶来训练Vertex AI。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用一个共享的测试账户或项目。为了避免用户在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，则您的环境已经经过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中键入"Vertex"，然后选择**Vertex管理员**。在过滤框中键入"Storage Object Admin"，然后选择**Storage Object Admin**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境中。

将您的服务帐户密钥的路径输入为下面单元格中的GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，都需要执行以下步骤。**

当您初始化用于Python的Vertex SDK时，您需要指定一个云存储的暂存桶。这个暂存桶是所有与您的数据集和模型资源相关的数据在会话之间保留的地方。

在下方设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶还不存在的情况下：运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查Cloud Storage桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设定变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化Python的Vertex SDK

为你的项目和对应的存储桶初始化Python的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在你已经准备好开始创建自己的AutoML表格二分类模型。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-tables-data/bank-marketing.csv"

快速查看您的数据

此教程使用存储在公共云存储桶中的银行营销数据集版本，使用CSV索引文件。

首先，对数据进行快速查看。通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

您还需要知道标签列的标题名称用于训练，该信息保存在`label_column`中。对于这个数据集，标签列在CSV文件的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

### 创建数据集

接下来，使用 `TabularDataset` 类的 `create` 方法创建 `Dataset` 资源，需要提供以下参数：

- `display_name`：`Dataset` 资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入到 `Dataset` 资源中。
- `bq_source`：或者，从 BigQuery 表中导入数据项到 `Dataset` 资源中。

此操作可能需要几分钟的时间。

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="Bank Marketing" + "_" + TIMESTAMP, gcs_source=[IMPORT_FILE]
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行管道。

#### 创建训练管道

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练管道，需指定以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `optimization_prediction_type`：训练模型的任务类型。
  - `classification`：表格分类模型。
  - `regression`：表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLTabularTrainingJob(
    display_name="bank_" + TIMESTAMP,
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

print(dag)

运行训练管道

接下来，您可以通过调用方法`run`来运行DAG，启动训练作业，参数如下：

- `dataset`：要训练模型的`Dataset`资源。
- `model_display_name`：经过训练的模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留置数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：要作为标签训练的列的名称。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则在服务认为不能进一步改进模型目标测量之前，训练可能在使用整个预算之前完成。

当完成`run`方法时，会返回`Model`资源。

训练管道的执行时间最长为20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="bank_" + TIMESTAMP,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

## 回顾模型评估分数
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取一个对新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，也可以列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=bank_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

## 部署模型

接下来，将您的模型部署用于在线预测。要部署模型，您可以调用`deploy`方法，并使用以下参数：

- `machine_type`：计算机类型。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送一个在线预测请求

发送一个在线预测请求给您部署的模型。

创建测试项目

您将使用合成数据作为测试数据项。不要担心我们使用的是合成数据 - 我们只是想要演示如何做出预测。

In [ ]:
INSTANCE = {
    "Age": "58",
    "Job": "managment",
    "MaritalStatus": "married",
    "Education": "teritary",
    "Default": "no",
    "Balance": "2143",
    "Housing": "yes",
    "Loan": "no",
    "Contact": "unknown",
    "Day": "5",
    "Month": "may",
    "Duration": "261",
    "Campaign": "1",
    "PDays": "-1",
    "Previous": "0",
    "POutcome": "unknown",
}

### 进行预测

现在您的`Model`资源已部署到一个`Endpoint`资源，您可以通过向`Endpoint`资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为:

    [feature_list]

由于predict()方法可以接受多个项目(实例)，请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

predict()调用的响应是一个Python字典，具有以下条目:

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `displayNames`: 每个类标签的类名。
- `confidences`: 每个类标签的预测置信度，介于0到1之间。
- `deployed_model_id`: 执行预测的已部署`Model`资源的Vertex AI标识符。

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.predict(instances_list)
print(prediction)

## 卸载模型

当您完成预测时，您需要从“端点”资源中卸载模型。这将取消所有计算资源，并停止为部署的模型计费。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理此项目中使用的所有 Google Cloud 资源，您可以删除用于教程的[Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- AutoML 训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud 存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME